In [1]:
import numpy as np
import string

In [2]:
np.random.seed(1234)

In [3]:
initial = {} # start of the phrase
first_order = {} # second word only
second_order = {}

In [5]:
def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))


In [6]:
import os

if not os.path.exists('robert_frost.txt'):
    !wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt
else:
    print('File is already downloaded.')

File is already downloaded.


In [7]:
def add2dict(d, k, v):
    if k not in d:
        d[k] = []
    d[k].append(v)

In [8]:
for line in open('robert_frost.txt'):
    tokens = remove_punctuation(line.rstrip().lower()).split()
    
    T = len(tokens)
    for i in range(T):
        t = tokens[i]
        if i == 0:
            # measure the distribution of the first word
            initial[t] = initial.get(t, 0.) + 1
        else:
            t_1 = tokens[i-1]
            if i == T - 1:
                # measure probability of ending the line
                add2dict(second_order, (t_1, t), 'END')
            if i == 1:
                # measure distribution of second word
                add2dict(first_order, t_1, t)
            else:
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

In [9]:
# normalize the distributions
initial_total = sum(initial.values())
for t, c in initial.items():
    initial[t] = c / initial_total

In [10]:
# Convert [cat, cat, cat, dog, dog, dog, mouse, ... ]
# into { cat: 0.5, dog: 0.5, mouse: 0.1, ... }

def list2pdict(ts):
    # Turn each list of possibilities into a dictionary to probabilities
    d = {}
    n = len(ts)
    for t in ts:
        d[t] = d.get(t, 0.) + 1
    for t, c in d.items():
        d[t] = c/n
    return d

In [11]:
for t_1, ts in first_order.items():
    # Replace list with dictionary of probabilities
    first_order[t_1] = list2pdict(ts)

In [12]:
for k, ts, in second_order.items():
    second_order[k] = list2pdict(ts)

In [14]:
def sample_word(d):
    p0 = np.random.random()
    cumulative = 0
    for t, p in d.items():
        cumulative += p
        if p0 < cumulative:
            return t
    assert False

            

In [16]:
def generate():
    for i in range(4):
        sentence = []
        
        # initial word
        w0 = sample_word(initial)
        sentence.append(w0)
        
        # sample second word
        w1 = sample_word(first_order[w0])
        sentence.append(w1)
        
        # second-order transitions until END
        while True:
            w2 = sample_word(second_order[(w0, w1)])
            if w2 == 'END':
                break
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))

In [20]:
generate()

dyou know a person so related to herself
you take the polish off the ground
no i dont follow you
in leaves no step had trodden black
